In [1]:
import warnings
warnings.filterwarnings('ignore')
import os

import tws_data 
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
from ib_insync import *
import numpy as np
import pandas as pd
import datetime as dt
import time
pd.options.display.float_format = '{:,.4f}'.format
%matplotlib inline
from scipy.stats import norm as norm
import matplotlib.pyplot as plt
import requests as req
from bs4 import BeautifulSoup as bs
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import math


In [2]:
os.chdir('D:\Options Data\IB Intraday')

def create_dfrow(curr_option):
    df_row = pd.DataFrame({'Symbol': curr_option.contract.localSymbol, 
                           'Type':curr_option.contract.right, 
                           'Bid': curr_option.bid, 
                           'Ask': curr_option.ask, 
                           'Volume': curr_option.volume, 
                           'Strike': curr_option.contract.strike,
                           'Expiry': dt.datetime.strptime(curr_option.contract.lastTradeDateOrContractMonth, '%Y%m%d'),
                           'bidIV': curr_option.bidGreeks.impliedVol, 
                           'bidDelta': curr_option.bidGreeks.delta,
                           'bidGamma': curr_option.bidGreeks.gamma, 
                           'bidVega': curr_option.bidGreeks.vega,
                           'bidTheta': curr_option.bidGreeks.theta, 
                           'askIV': curr_option.askGreeks.impliedVol, 
                           'askDelta': curr_option.askGreeks.delta,
                           'askGamma': curr_option.askGreeks.gamma, 
                           'askVega': curr_option.askGreeks.vega,
                           'askTheta': curr_option.askGreeks.theta,
                           'modelIV': curr_option.modelGreeks.impliedVol, 
                           'modelDelta': curr_option.modelGreeks.delta,
                           'modelGamma': curr_option.modelGreeks.gamma, 
                           'modelVega': curr_option.modelGreeks.vega,
                           'modelTheta': curr_option.modelGreeks.theta}, index = [0])
    return df_row

def get_movers():

    indexarb_url = 'http://www.indexarb.com/stockVsIndexesMiddleColumn.html#djIndexAnalysis'
    indexarb_soup = bs(req.get(indexarb_url).text,'lxml')
    indexarb_tables = indexarb_soup.select('table[align="LEFT"]')

    movers = []

    for i, table_html in enumerate(indexarb_tables):
        curr_table = pd.read_html(str(table_html))[2]
        curr_table.columns = curr_table.loc[1,:].tolist()

        if i == 0:
            curr_table = curr_table.iloc[2:,:4].set_index('Index').apply(pd.to_numeric)
        else:
            curr_table = curr_table.iloc[2:,[1,3,4,5,6]].dropna().set_index('Sym').apply(pd.to_numeric)

        curr_table.columns = [x.split(' ')[0] for x in curr_table.columns.tolist()]
        movers.append(curr_table)

    index = movers[0]
    movers = pd.concat(movers[1:], axis = 0)
    movers = movers.sort_values('OneMonth')
    
    return index, movers

def time_remaining(weekly_options, curr_spx):
    typ = weekly_options.reset_index().loc[0,'Type']

    try:
        interest_rate = bs(req.get('https://www.marketwatch.com/investing/bond/tmubmusd01m?countrycode=bx','lxml').text, 'lxml')
        interest_rate = float(interest_rate.select_one('h3[class*="intraday__price"]').select_one('bg-quote').text)/100
    except:
        interest_rate = 0.02422

    weekly_options['a'] = interest_rate - (weekly_options['modelIV']**2)/2

    if typ == 'P':
        weekly_options['b'] = weekly_options['modelIV']*norm.ppf(-weekly_options['modelDelta'])
    if typ == 'C':
        weekly_options['b'] = -weekly_options['modelIV']*norm.ppf(weekly_options['modelDelta'])
    weekly_options['c'] = np.log(curr_spx/weekly_options['Strike'])

    weekly_options_otm = weekly_options[abs(weekly_options.modelDelta) <= 0.5]
    weekly_options_itm = weekly_options[abs(weekly_options.modelDelta) > 0.5]

    if typ == 'P':
        weekly_options_otm['time_remaining'] = ((-weekly_options_otm.b - 
                                                 np.sqrt(weekly_options_otm.b**2 - 
                                                         4*weekly_options_otm.a*weekly_options_otm.c))/(2*weekly_options_otm.a))**2
        weekly_options_itm['time_remaining'] = ((-weekly_options_itm.b + 
                                                 np.sqrt(weekly_options_itm.b**2 - 
                                                         4*weekly_options_itm.a*weekly_options_itm.c))/(2*weekly_options_itm.a))**2
    if typ == 'C':
        weekly_options_otm['time_remaining'] = ((-weekly_options_otm.b + 
                                                 np.sqrt(weekly_options_otm.b**2 - 
                                                         4*weekly_options_otm.a*weekly_options_otm.c))/(2*weekly_options_otm.a))**2
        weekly_options_itm['time_remaining'] = ((-weekly_options_itm.b - 
                                                 np.sqrt(weekly_options_itm.b**2 - 
                                                         4*weekly_options_itm.a*weekly_options_itm.c))/(2*weekly_options_itm.a))**2
    weekly_times = pd.concat([weekly_options_otm, 
                              weekly_options_itm], axis = 0)[['Expiry','time_remaining']].dropna().groupby('Expiry').mean()

    del weekly_options['a'], weekly_options['b'], weekly_options['c'], weekly_options_itm, weekly_options_otm

    return weekly_options.merge(weekly_times.reset_index(), on = 'Expiry')


def get_options(ticker, exchange, contractType, earliest_date, latest_date, curr_iv, stdevs, save_csv = False):
    if contractType == 'Stock':
        security = Stock(ticker,exchange)
    else:
        security = Index(ticker, exchange)

    ib.qualifyContracts(security)

    chains = ib.reqSecDefOptParams(security.symbol, '', security.secType, security.conId)

    chains_df = util.df(chains)
    chain = next(c for c in chains if c.tradingClass == ticker and c.exchange == 'SMART')

    all_expirations = sorted(exp for exp in chain.expirations)
    expirations_dates = [dt.datetime.strptime(x, '%Y%m%d') for x in all_expirations]

    expirations = list(filter(lambda x: x[0] >= earliest_date and x[0] <= latest_date, list(zip(expirations_dates,all_expirations))))
    dte_expirations = [(expir[0] - dt.datetime.today()).days for expir in expirations]
    expirations = [expir[1] for expir in expirations]

    ib.reqHeadTimeStamp(security, whatToShow='TRADES', useRTH=True)

    end_date = '' #'20100506 13:00:00 PST'
    duration = '1 D'

    sec_bars = ib.reqHistoricalData(
            security,
            endDateTime=end_date,
            durationStr=duration,
            barSizeSetting='1 min',
            whatToShow='TRADES',
            useRTH=True,
            formatDate=1)


    sec_df = util.df(sec_bars).set_index('date')
    sec_df.index = pd.to_datetime(sec_df.index)

    curr_sec_price = sec_df.reset_index().loc[len(sec_df) - 1, 'close']
    bounds_lst = [curr_sec_price*(stdevs*stdevs/(100*np.sqrt(365.25/dte))) for dte in dte_expirations]


    contracts_list = []

    for bounds, expiration in zip(bounds_lst, expirations):
#         strikes = [strike for strike in chain.strikes
#                    if strike % 5 == 0
#                    and curr_sec_price - bounds < strike < curr_sec_price + bounds]
        strikes = [strike for strike in chain.strikes
                   if curr_sec_price - bounds < strike < curr_sec_price + bounds]

        rights = ['P', 'C']

        curr_contracts = [Option(ticker, expiration, strike, right, 'SMART')
                          for right in rights
                          for strike in strikes]
        contracts_list += [curr_contracts]

    puts_list = []
    calls_list = []

    for contracts in contracts_list:
        ib.qualifyContracts(*contracts)
        sec_options = ib.reqTickers(*contracts)

        puts_rows = []
        calls_rows = []

        for curr_option in sec_options:
            try:
                curr_row = create_dfrow(curr_option)
                if curr_option.contract.right == 'P':
                    puts_rows.append(curr_row)
                elif curr_option.contract.right == 'C':
                    calls_rows.append(curr_row)
            except:
                continue

        puts = pd.concat(puts_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])
        calls = pd.concat(calls_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])
        puts = time_remaining(puts, curr_sec_price)
        calls = time_remaining(calls, curr_sec_price)
        puts_list.append(puts)
        calls_list.append(calls)
        time.sleep(5)

    puts_df = pd.concat(puts_list,axis = 0).reset_index(drop = True)
    calls_df = pd.concat(calls_list,axis = 0).reset_index(drop = True)

    if save_csv:
        curr_time = dt.datetime.now().strftime('%Y%m%d-%H-%M')
        puts_df.to_csv('ib_puts' + curr_time + '.csv')
        calls_df.to_csv('ib_calls' + curr_time + '.csv')
        
    return puts_df, calls_df, curr_sec_price


def get_spreads(weekly_options, curr_spx, expiry_index = 0, rowshift = 1, commissions = 0.0266):

    expirations_dates = weekly_options.Expiry.drop_duplicates().tolist()

    weekly_options = weekly_options[weekly_options.Expiry == expirations_dates[expiry_index]]

    typ = weekly_options.reset_index().loc[0,'Type']

    if typ == 'P':
        weekly_options = weekly_options.sort_values('Strike', ascending = False).reset_index(drop = True)
    if typ == 'C':
        weekly_options = weekly_options.sort_values('Strike', ascending = True).reset_index(drop = True)

    time_remaining = weekly_options.reset_index(drop = True).loc[0,'time_remaining']

    weekly_options['Mid'] = (weekly_options.Ask + weekly_options.Bid)/2
    shorts = weekly_options[['Mid', 'Strike', 'bidDelta', 'bidGamma', 'bidTheta', 'bidVega']]
    shorts.columns = ['Price','Strike','Delta','Gamma','Theta','Vega']
    longs = weekly_options[['Mid','Strike', 'askDelta', 'askGamma','askTheta', 'askVega']]
    longs.columns = ['Price','Strike','Delta','Gamma','Theta','Vega']

    spreads = shorts - longs.shift(-rowshift)
    spreads['Short_Strike'] = shorts.Strike
    spreads['Short_Delta'] = shorts.Delta
    spreads['Long_Strike'] = longs.Strike.shift(-rowshift)
    spreads = spreads[['Short_Strike', 'Long_Strike', 'Short_Delta', 'Price',
                       'Strike', 'Delta', 'Gamma', 'Theta', 'Vega']]

    spreads['Credit'] = spreads.Price - commissions

    if typ == 'P':
        spreads['MaxLoss'] = (-spreads.Strike + spreads.Credit)*100
        spreads['BreakEven'] = spreads['Short_Strike'] - spreads.Credit
    if typ == 'C':
        spreads['MaxLoss'] = (spreads.Strike + spreads.Credit)*100
        spreads['BreakEven'] = spreads['Short_Strike'] + spreads.Credit

    del spreads['Price'], spreads['Strike']

    spreads = spreads[spreads.Credit > 0].dropna().reset_index(drop = True)
    spreads['Expiry'] = expirations_dates[expiry_index]

    return spreads

def art_hedges(curr_calls, curr_puts, curr_price, expiry_index, bias = 'upside',
               maxshift = 6, commissions = 0.0266):
    
    hedge = []
    for rowshift in range(1,maxshift):

        if bias == 'upside':
            spreads = get_spreads(curr_puts, curr_price, expiry_index, rowshift, commissions = 0.0266)
        else:
            spreads = get_spreads(curr_calls, curr_price, expiry_index, rowshift, commissions = 0.0266)
        spreads = spreads[(spreads.Credit > 0.5)]
        hedge.append(spreads)

    hedge = pd.concat(hedge, axis = 0).reset_index(drop = True)

    if bias == 'upside':
        expirations_dates = curr_calls.Expiry.drop_duplicates().tolist()

        long_hedge = curr_calls[(curr_calls.Expiry == expirations_dates[expiry_index])][['Ask', 'Bid', 'Expiry', 'Strike',
                                                                             'modelDelta', 'modelGamma',
                                                                             'modelTheta', 'modelVega', 'time_remaining']].reset_index(drop = True)
    else:
        expirations_dates = curr_puts.Expiry.drop_duplicates().tolist()

        long_hedge = curr_puts[(curr_puts.Expiry == expirations_dates[expiry_index])][['Ask', 'Bid', 'Expiry', 'Strike',
                                                                             'modelDelta', 'modelGamma',
                                                                             'modelTheta', 'modelVega', 'time_remaining']].reset_index(drop = True)

    long_hedge['Mid'] = (long_hedge['Ask'] + long_hedge['Bid'])/2
    long_hedge.columns = [x.replace('model','') for x in long_hedge.columns.tolist()]

    hedges_lst = []

    for idx, row in hedge.iterrows():

        curr_hedge = long_hedge[long_hedge.Mid < row.Credit][['Mid','Expiry', 'Strike',
                                                              'Delta', 'Gamma',
                                                              'Theta', 'Vega', 'time_remaining']]
        curr_hedge['Gamma'] = curr_hedge['Gamma'] - row.Gamma
        curr_hedge['Delta'] = curr_hedge['Delta'] - row.Delta
        curr_hedge['Theta'] = curr_hedge['Theta'] - row.Theta
        curr_hedge['Vega'] = curr_hedge['Vega'] - row.Vega
        if bias == 'upside':
            curr_hedge = curr_hedge[(curr_hedge.Delta > 0) &
                                    (curr_hedge.Gamma > 0) &
                                    (curr_hedge.Theta > 0) &
                                    (curr_hedge.Vega > 0)]
        else:
            curr_hedge = curr_hedge[(curr_hedge.Delta < 0) &
                                (curr_hedge.Gamma > 0) &
                                (curr_hedge.Theta > 0) &
                                (curr_hedge.Vega > 0)]
        curr_hedge['Credit'] = -curr_hedge.Mid + row.Credit - 0.0133

        if bias == 'upside':
            curr_hedge['MaxLoss'] = (row.Long_Strike - row.Short_Strike + curr_hedge.Credit)*100
            curr_hedge['BreakEven'] = row.Short_Strike - curr_hedge['Credit']
        else:
            curr_hedge['MaxLoss'] = (row.Short_Strike - row.Long_Strike + curr_hedge.Credit)*100
            curr_hedge['BreakEven'] = row.Short_Strike + curr_hedge['Credit']
        curr_hedge['Maturity'] = curr_hedge['Expiry']
        curr_hedge['Mid'] = row.Short_Strike
        curr_hedge['Expiry'] = row.Long_Strike

        if bias == 'upside':
            curr_hedge.columns = ['Short_Put_Strike','Long_Put_Strike','Long_Call_Strike'] + curr_hedge.columns.tolist()[3:]
        else:
            curr_hedge.columns = ['Short_Call_Strike','Long_Call_Strike','Long_Put_Strike'] + curr_hedge.columns.tolist()[3:]

        hedges_lst.append(curr_hedge)

    hedges_df = pd.concat(hedges_lst, axis = 0)
    
    return hedges_df

In [3]:
index, movers = get_movers()

In [4]:
losers = movers.head()
winners = movers.tail().sort_values('OneMonth', ascending = False)

In [5]:
client_id = 12

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=client_id)

<IB connected to 127.0.0.1:7496 clientId=12>

In [4]:
# exchange = 'NYSE'
# contractType = 'Stock'
# earliest_date = dt.datetime(2019,4,6)
# latest_date = dt.datetime(2019,6,30)
# curr_iv = 40
# stdevs = 9

# stock_prices = {}
# stock_puts = {}
# stock_calls = {}

# for ticker in losers.index:

#     curr_puts, curr_calls, curr_price = get_options(ticker, exchange, contractType, earliest_date, latest_date, curr_iv, stdevs, save_csv = False)
    
#     stock_prices[ticker] = curr_price
#     stock_puts[ticker] = curr_puts
#     stock_calls[ticker] = curr_calls
#     break

In [23]:
exchange = 'NYSE'
contractType = 'Stock'
earliest_date = dt.datetime(2020,1,3)
latest_date = dt.datetime(2020,4,30)

ticker = 'MSFT'
curr_iv = 30
stdevs = 30
curr_puts, curr_calls, curr_price = get_options(ticker, exchange, contractType, earliest_date, latest_date, curr_iv, stdevs, save_csv = False)

commissionReport: No execution found for CommissionReport(execId='000187e8.5d778707.02.01', commission=3.8487, currency='USD')
commissionReport: No execution found for CommissionReport(execId='000187e8.5d778707.03.01', commission=1.2829, currency='USD')
commissionReport: No execution found for CommissionReport(execId='000187e8.5d778707.04.01', commission=5.154918, currency='USD')
commissionReport: No execution found for CommissionReport(execId='000187e8.5d778707.05.01', commission=3.8487, currency='USD')
commissionReport: No execution found for CommissionReport(execId='000187e8.5d778707.06.01', commission=1.2829, currency='USD')
commissionReport: No execution found for CommissionReport(execId='000187e8.5d778708.02.01', commission=0.4929, currency='USD')
commissionReport: No execution found for CommissionReport(execId='000187e8.5d778708.03.01', commission=0.498729, currency='USD')
commissionReport: No execution found for CommissionReport(execId='000187e8.5d778708.04.01', commission=0.49

Error 200, reqId 993: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=118.0, right='P', exchange='SMART')
Error 200, reqId 994: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=119.0, right='P', exchange='SMART')
Error 200, reqId 996: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=121.0, right='P', exchange='SMART')
Error 200, reqId 998: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=123.0, right='P', exchange='SMART')
Error 200, reqId 997: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=122.0, right='P', exchange='SMART')
Error 200, reqId 999: No security d

Error 200, reqId 1076: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=127.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=128.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=129.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=131.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=132.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=133.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=134.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContrac

Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=119.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=121.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=122.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=123.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=124.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=126.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=127.0, right='C', exchange='SMART')
Started to throttle requests
Error 200, reqId 1077: No security definition has been found for the request, contract: Option(symbol='

Error 200, reqId 1154: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=131.0, right='C', exchange='SMART')
Error 200, reqId 1155: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=132.0, right='C', exchange='SMART')
Error 200, reqId 1156: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=133.0, right='C', exchange='SMART')
Error 200, reqId 1157: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=134.0, right='C', exchange='SMART')
Error 200, reqId 1159: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200117', strike=136.0, right='C', exchange='SMART')
Error 200, reqId 1160: No secu

...

Error 200, reqId 1600: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=133.0, right='C', exchange='SMART')
Error 200, reqId 1601: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=134.0, right='C', exchange='SMART')
Error 200, reqId 1603: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=136.0, right='C', exchange='SMART')
Error 200, reqId 1604: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=137.0, right='C', exchange='SMART')
Error 200, reqId 1605: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=138.0, right='C', exchange='SMART')
Error 200, reqId 1606: No secu

Error 200, reqId 1667: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=126.0, right='C', exchange='SMART')
Error 200, reqId 1668: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=127.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=128.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=129.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=131.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=132.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=133.0, right='P', exchange='

Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=147.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=148.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=149.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=152.5, right='C', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=157.5, right='C', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=37.5, right='C', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=40.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=42.5, right='C', exchange='SMART')
Unknown con

Error 200, reqId 1698: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=40.0, right='P', exchange='SMART')
Error 200, reqId 1699: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=42.5, right='P', exchange='SMART')
Error 200, reqId 1701: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=45.0, right='P', exchange='SMART')
Error 200, reqId 1702: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=47.5, right='P', exchange='SMART')
Error 200, reqId 1704: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=50.0, right='P', exchange='SMART')
Error 200, reqId 1706: No security 

Error 200, reqId 1762: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=147.0, right='C', exchange='SMART')
Error 200, reqId 1763: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=148.0, right='C', exchange='SMART')
Stopped to throttle requests
Error 200, reqId 1764: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=149.0, right='C', exchange='SMART')
Error 200, reqId 1766: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=152.5, right='C', exchange='SMART')
Error 200, reqId 1768: No security definition has been found for the request, contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20200417', strike=157.5, right='C', exchange='SMART')
E

In [24]:
maxshift = 30

arteHedges_upside = []
for expiry_index in range(len(curr_calls.Expiry.drop_duplicates())):
    bias = 'upside'
    possible_pos = art_hedges(curr_calls, curr_puts, curr_price, expiry_index, bias,
                              maxshift = maxshift, commissions = 0.0266)
    arteHedges_upside.append(possible_pos)

arteHedges_upside = pd.concat(arteHedges_upside,axis = 0)

In [25]:
arteHedges_downside = []
for expiry_index in range(len(curr_calls.Expiry.drop_duplicates())):
    bias = 'downside'
    possible_pos = art_hedges(curr_calls, curr_puts, curr_price, expiry_index, bias,
                              maxshift = maxshift, commissions = 0.0266)
    arteHedges_downside.append(possible_pos)
arteHedges_downside = pd.concat(arteHedges_downside,axis = 0)

arteHedges_upside['ReturnOnRisk'] = abs(arteHedges_upside.Credit/arteHedges_upside.MaxLoss)
arteHedges_downside['ReturnOnRisk'] = abs(arteHedges_downside.Credit/arteHedges_downside.MaxLoss)

In [26]:
potential_ups = arteHedges_upside[(arteHedges_upside.Short_Put_Strike < curr_price) &
                                  (arteHedges_upside.Short_Put_Strike > 0.001)].sort_values(['Maturity', 
                                                                                             'ReturnOnRisk'], ascending = False)
try:
    potential_ups['downmove_to_breach'] = potential_ups['Short_Put_Strike']/curr_price - 1
    potential_ups['upmove_to_win'] = potential_ups['Long_Call_Strike']/curr_price - 1
    potential_ups = potential_ups.sort_values('MaxLoss').reset_index(drop = True)
    potential_ups['MaxLossScore'] = potential_ups.index
    
    potential_ups = potential_ups.sort_values('downmove_to_breach', ascending = False).reset_index(drop = True)
    potential_ups['DownMoveScore'] = potential_ups.index
    
    potential_ups = potential_ups.sort_values('upmove_to_win', ascending = False).reset_index(drop = True)
    potential_ups['UpMoveScore'] = potential_ups.index
    
    potential_ups['Score'] = potential_ups.MaxLossScore + potential_ups.DownMoveScore + potential_ups.UpMoveScore
    potential_ups = potential_ups.sort_values('Score', ascending = False)
except:
    None
potential_ups

,Short_Put_Strike,Long_Put_Strike,Long_Call_Strike,Delta,Gamma,Theta,Vega,time_remaining,Credit,MaxLoss,BreakEven,Maturity,ReturnOnRisk,downmove_to_breach,upmove_to_win,MaxLossScore,DownMoveScore,UpMoveScore,Score
79,97.5000,70.0000,165.0000,0.1000,0.0040,0.0001,0.0217,0.3058,0.0951,"-2,740.4900",97.4049,2020-01-17,0.0000,-0.2763,0.2248,117,103,79,299
88,105.0000,85.0000,165.0000,0.1193,0.0028,0.0008,0.0135,0.3058,0.3551,"-1,964.4900",104.6449,2020-01-17,0.0002,-0.2206,0.2248,136,66,88,290
80,97.5000,65.0000,165.0000,0.1018,0.0038,0.0006,0.0212,0.3058,0.1201,"-3,237.9900",97.3799,2020-01-17,0.0000,-0.2763,0.2248,107,102,80,289
86,100.0000,75.0000,165.0000,0.1052,0.0037,0.0004,0.0293,0.3058,0.1901,"-2,480.9900",99.8099,2020-01-17,0.0001,-0.2577,0.2248,121,80,86,287
89,110.0000,95.0000,165.0000,0.1318,0.0023,0.0003,0.0210,0.3058,0.5351,"-1,446.4900",109.4649,2020-01-17,0.0004,-0.1835,0.2248,145,51,89,285
85,105.0000,80.0000,165.0000,0.1251,0.0023,0.0020,0.0054,0.3058,0.4351,"-2,456.4900",104.5649,2020-01-17,0.0002,-0.2206,0.2248,123,75,85,283
87,105.0000,82.5000,165.0000,0.1227,0.0025,0.0015,0.0129,0.3058,0.4001,"-2,209.9900",104.5999,2020-01-17,0.0002,-0.2206,0.2248,132,63,87,282
121,105.0000,70.0000,160.0000,0.1707,0.0049,0.0002,0.0517,0.3058,0.1901,"-3,480.9900",104.8099,2020-01-17,0.0001,-0.2206,0.1876,101,59,121,281
30,80.0000,42.5000,175.0000,0.0307,0.0020,0.0004,0.0214,0.3058,0.5051,"-3,699.4900",79.4949,2020-01-17,0.0001,-0.4062,0.2990,94,152,30,276
24,82.5000,47.5000,175.0000,0.0320,0.0018,0.0002,0.0173,0.3058,0.5301,"-3,446.9900",81.9699,2020-01-17,0.0002,-0.3876,0.2990,103,147,24,274


In [27]:
potential_downs = arteHedges_downside[(arteHedges_downside.Short_Call_Strike > curr_price)].sort_values(['Maturity', 
                                                                                                         'Long_Put_Strike'], ascending = False)
try:
    potential_downs['upmove_to_breach'] = potential_downs['Short_Call_Strike']/curr_price - 1
    potential_downs['downmove_to_win'] = potential_downs['Long_Put_Strike']/curr_price - 1
    potential_downs = potential_downs.sort_values('MaxLoss').reset_index(drop = True)
    potential_downs['MaxLossScore'] = potential_downs.index
    
    potential_downs = potential_downs.sort_values('upmove_to_breach').reset_index(drop = True)
    potential_downs['UpMoveScore'] = potential_downs.index
    
    potential_downs = potential_downs.sort_values('downmove_to_win').reset_index(drop = True)
    potential_downs['DownMoveScore'] = potential_downs.index
    
    potential_downs['Score'] = potential_downs.MaxLossScore + potential_downs.UpMoveScore + potential_downs.DownMoveScore
    potential_downs = potential_downs.sort_values('Score', ascending = False)
except:
    None
potential_downs

,Short_Call_Strike,Long_Call_Strike,Long_Put_Strike,Delta,Gamma,Theta,Vega,time_remaining,Credit,MaxLoss,BreakEven,Maturity,ReturnOnRisk,upmove_to_breach,downmove_to_win,MaxLossScore,UpMoveScore,DownMoveScore,Score
36,135.0000,180.0000,130.0000,-0.8874,0.0027,0.0016,0.0712,0.3752,2.0101,"-4,298.9900",137.0101,2020-03-20,0.0005,0.0021,-0.0350,17,36,36,89
35,135.0000,180.0000,130.0000,-0.8806,0.0034,0.0005,0.1126,0.4535,1.9601,"-4,303.9900",136.9601,2020-04-17,0.0005,0.0021,-0.0350,16,35,35,86
22,140.0000,180.0000,125.0000,-0.7293,0.0005,0.0006,0.0495,0.3752,1.3101,"-3,868.9900",141.3101,2020-03-20,0.0003,0.0392,-0.0721,21,41,22,84
14,140.0000,175.0000,120.0000,-0.6470,0.0000,0.0001,0.0464,0.3752,2.5651,"-3,243.4900",142.5651,2020-03-20,0.0008,0.0392,-0.1093,27,39,14,80
25,140.0000,185.0000,125.0000,-0.7435,0.0004,0.0010,0.0550,0.4535,1.4151,"-4,358.4900",141.4151,2020-04-17,0.0003,0.0392,-0.0721,15,40,25,80
0,135.0000,140.0000,55.0000,-0.1043,0.0004,0.0008,0.0016,0.2960,2.3901,-260.9900,137.3901,2020-01-17,0.0092,0.0021,-0.5917,41,38,0,79
33,135.0000,185.0000,130.0000,-0.8932,0.0023,0.0017,0.0776,0.4535,2.0901,"-4,790.9900",137.0901,2020-04-17,0.0004,0.0021,-0.0350,11,31,33,75
19,135.0000,180.0000,125.0000,-0.8106,0.0019,0.0009,0.0862,0.4535,3.7351,"-4,126.4900",138.7351,2020-04-17,0.0009,0.0021,-0.0721,19,37,19,75
38,135.0000,190.0000,135.0000,-0.9776,0.0026,0.0029,0.0664,0.4535,0.0151,"-5,498.4900",135.0151,2020-04-17,0.0000,0.0021,0.0021,7,27,38,72
39,135.0000,190.0000,135.0000,-0.9866,0.0021,0.0041,0.0573,0.3752,0.0101,"-5,498.9900",135.0101,2020-03-20,0.0000,0.0021,0.0021,6,26,39,71


In [30]:
potential_ups[(potential_ups.Long_Call_Strike < 165) &
              (potential_ups.MaxLoss > -2500)]

,Short_Put_Strike,Long_Put_Strike,Long_Call_Strike,Delta,Gamma,Theta,Vega,time_remaining,Credit,MaxLoss,BreakEven,Maturity,ReturnOnRisk,downmove_to_breach,upmove_to_win,MaxLossScore,DownMoveScore,UpMoveScore,Score
113,115.0000,92.5000,160.0000,0.2200,0.0025,0.0007,0.0210,0.3058,0.9051,"-2,159.4900",114.0949,2020-01-17,0.0004,-0.1464,0.1876,133,23,113,269
98,110.0000,85.0000,160.0000,0.1904,0.0039,0.0001,0.0400,0.3058,0.4601,"-2,453.9900",109.5399,2020-01-17,0.0002,-0.1835,0.1876,124,44,98,266
116,120.0000,100.0000,160.0000,0.2549,0.0013,0.0004,0.0079,0.3058,1.4901,"-1,850.9900",118.5099,2020-01-17,0.0008,-0.1093,0.1876,140,8,116,264
110,115.0000,90.0000,160.0000,0.2260,0.0021,0.0017,0.0200,0.3058,0.9851,"-2,401.4900",114.0149,2020-01-17,0.0004,-0.1464,0.1876,127,20,110,257
101,120.0000,95.0000,160.0000,0.2711,0.0002,0.0027,0.0050,0.3058,1.7101,"-2,328.9900",118.2899,2020-01-17,0.0007,-0.1093,0.1876,129,4,101,234
91,120.0000,97.5000,160.0000,0.2630,0.0007,0.0016,0.0062,0.3058,1.6151,"-2,088.4900",118.3849,2020-01-17,0.0008,-0.1093,0.1876,134,3,91,228


In [65]:
ib.disconnect()